# HuggingFace

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/translation-huggingface](https://github.com/huseinzol05/Malaya/tree/master/example/translation-huggingface).
    
</div>

<div class="alert alert-info">

This module trained on both standard and local (included social media) language structures, so it is save to use for both.
    
</div>

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
%%time

import malaya
import logging

logging.basicConfig(level=logging.INFO)

Cannot import beam_search_ops from Tensorflow 1, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] for stemmer will not available to use, make sure Tensorflow 1 version >= 1.15


CPU times: user 4.94 s, sys: 3.31 s, total: 8.25 s
Wall time: 4.35 s


### List available HuggingFace models

In [3]:
malaya.translation.available_huggingface()

INFO:malaya.translation:tested on FLORES200 pair `dev` set, https://github.com/huseinzol05/malay-dataset/tree/master/translation/flores200-eval
INFO:malaya.translation:tested on noisy test set, https://github.com/huseinzol05/malay-dataset/tree/master/translation/noisy-eval
INFO:malaya.translation:check out NLLB 200 metrics from `malaya.translation.nllb_metrics`.
INFO:malaya.translation:check out Google Translate metrics from `malaya.translation.google_translate_metrics`.


,Size (MB),Suggested length,en-ms chrF2++,ms-en chrF2++,pasar ms-ms chrF2++,pasar ms-en chrF2++,from lang,to lang,old model,ind-ms chrF2++,jav-ms chrF2++,manglish-ms chrF2++,manglish-en chrF2++
mesolitica/finetune-noisy-translation-t5-tiny-bahasa-cased-v3,139,1024,43.937298,43.937298,43.937298,43.937298,"[en, ms, pasar ms]","[ms, en]",True,NaN,NaN,NaN,NaN
mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3,242,1024,43.937298,43.937298,43.937298,43.937298,"[en, ms, pasar ms]","[ms, en]",True,NaN,NaN,NaN,NaN
mesolitica/finetune-noisy-translation-t5-base-bahasa-cased-v3,892,1024,43.937298,43.937298,43.937298,43.937298,"[en, ms, pasar ms]","[ms, en]",True,NaN,NaN,NaN,NaN
mesolitica/translation-t5-tiny-standard-bahasa-cased,139,1536,43.937298,43.937298,43.937298,43.937298,"[en, ms, ind, jav, bjn, manglish, pasar ms]","[en, ms]",False,43.937298,43.937298,43.937298,43.937298
mesolitica/translation-t5-small-standard-bahasa-cased,242,1536,43.937298,43.937298,43.937298,43.937298,"[en, ms, ind, jav, bjn, manglish, pasar ms]","[en, ms]",False,43.937298,43.937298,43.937298,43.937298
mesolitica/translation-t5-base-standard-bahasa-cased,892,1536,43.937298,43.937298,43.937298,43.937298,"[en, ms, ind, jav, bjn, manglish, pasar ms]","[en, ms]",False,43.937298,43.937298,43.937298,43.937298


### Improvements of new model

1. able to translate `[en, ms, ind, jav, bjn, manglish, pasar ms]` while old model only able to translate `[en, ms, pasar ms]`.
2. No longer required `from_lang` part of the prefix.
3. able to retain text structure as it is.

### Load Transformer models

```python
def huggingface(
    model: str = 'mesolitica/translation-t5-small-standard-bahasa-cased',
    force_check: bool = True,
    from_lang: List[str] = None,
    to_lang: List[str] = None,
    old_model: bool = False,
    **kwargs,
):
    """
    Load HuggingFace model to translate.

    Parameters
    ----------
    model: str, optional (default='mesolitica/translation-t5-small-standard-bahasa-cased')
        Check available models at `malaya.translation.available_huggingface()`.
    force_check: bool, optional (default=True)
        Force check model one of malaya model.
        Set to False if you have your own huggingface model.

    Returns
    -------
    result: malaya.torch_model.huggingface.Translation
    """
```

In [4]:
model = malaya.translation.huggingface()

In [5]:
old_model = malaya.translation.huggingface(model = 'mesolitica/finetune-noisy-translation-t5-small-bahasa-cased-v3')

### Translate

```python
def generate(self, strings: List[str], from_lang: str = None, to_lang: str = 'ms', **kwargs):
    """
    Generate texts from the input.

    Parameters
    ----------
    strings : List[str]
    from_lang: str, optional (default=None)
        old model required `from_lang` parameter to make it works properly,
        while new model not required.
    to_lang: str, optional (default='ms')
        target language to translate.
    **kwargs: vector arguments pass to huggingface `generate` method.
        Read more at https://huggingface.co/docs/transformers/main_classes/text_generation

    Returns
    -------
    result: List[str]
    """
```

In [6]:
from pprint import pprint

In [7]:
# https://www.sinarharian.com.my/article/89678/BERITA/Politik/Saya-tidak-mahu-sentuh-isu-politik-Muhyiddin

string_news1 = 'TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh mengenai isu politik buat masa ini, sebaliknya mahu menumpukan kepada soal kebajikan rakyat serta usaha merancakkan semula ekonomi negara yang terjejas berikutan pandemik Covid-19. Perdana Menteri menjelaskan perkara itu ketika berucap pada Majlis Bertemu Pemimpin bersama pemimpin masyarakat Dewan Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.'
pprint(string_news1)

('TANGKAK - Tan Sri Muhyiddin Yassin berkata, beliau tidak mahu menyentuh '
 'mengenai isu politik buat masa ini, sebaliknya mahu menumpukan kepada soal '
 'kebajikan rakyat serta usaha merancakkan semula ekonomi negara yang terjejas '
 'berikutan pandemik Covid-19. Perdana Menteri menjelaskan perkara itu ketika '
 'berucap pada Majlis Bertemu Pemimpin bersama pemimpin masyarakat Dewan '
 'Undangan Negeri (DUN) Gambir di Dewan Serbaguna Bukit Gambir hari ini.')


In [8]:
# https://www.sinarharian.com.my/article/90021/BERITA/Politik/Tun-Mahathir-Anwar-disaran-bersara-untuk-selesai-kemelut-politik

string_news2 = 'ALOR SETAR - Kemelut politik Pakatan Harapan (PH) belum berkesudahan apabila masih gagal memuktamadkan calon Perdana Menteri yang dipersetujui bersama. Ahli Parlimen Sik, Ahmad Tarmizi Sulaiman berkata, sehubungan itu pihaknya mencadangkan mantan Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu), Tun Dr Mahathir Mohamad dan Presiden Parti Keadilan Rakyat (PKR), Datuk Seri Anwar Ibrahim mengundurkan diri daripada politik sebagai jalan penyelesaian.'
pprint(string_news2)

('ALOR SETAR - Kemelut politik Pakatan Harapan (PH) belum berkesudahan apabila '
 'masih gagal memuktamadkan calon Perdana Menteri yang dipersetujui bersama. '
 'Ahli Parlimen Sik, Ahmad Tarmizi Sulaiman berkata, sehubungan itu pihaknya '
 'mencadangkan mantan Pengerusi Parti Pribumi Bersatu Malaysia (Bersatu), Tun '
 'Dr Mahathir Mohamad dan Presiden Parti Keadilan Rakyat (PKR), Datuk Seri '
 'Anwar Ibrahim mengundurkan diri daripada politik sebagai jalan penyelesaian.')


In [9]:
string_news3 = 'Menteri Kanan (Kluster Keselamatan) Datuk Seri Ismail Sabri Yaakob berkata, kelonggaran itu diberi berikutan kerajaan menyedari masalah yang dihadapi mereka untuk memperbaharui dokumen itu. Katanya, selain itu, bagi rakyat asing yang pas lawatan sosial tamat semasa Perintah Kawalan Pergerakan (PKP) pula boleh ke pejabat Jabatan Imigresen yang terdekat untuk mendapatkan lanjutan tempoh.'
pprint(string_news3)

('Menteri Kanan (Kluster Keselamatan) Datuk Seri Ismail Sabri Yaakob berkata, '
 'kelonggaran itu diberi berikutan kerajaan menyedari masalah yang dihadapi '
 'mereka untuk memperbaharui dokumen itu. Katanya, selain itu, bagi rakyat '
 'asing yang pas lawatan sosial tamat semasa Perintah Kawalan Pergerakan (PKP) '
 'pula boleh ke pejabat Jabatan Imigresen yang terdekat untuk mendapatkan '
 'lanjutan tempoh.')


In [10]:
# https://qcikgubm.blogspot.com/2018/02/contoh-soalan-dan-jawapan-karangan.html

string_karangan = 'Selain itu, pameran kerjaya membantu para pelajar menentukan kerjaya yang akan diceburi oleh mereka. Seperti yang kita ketahui, pasaran kerjaya di Malaysia sangat luas dan masih banyak sektor pekerjaan di negara ini yang masih kosong kerana sukar untuk mencari tenaga kerja yang benar-benar berkelayakan. Sebagai contohnya, sektor perubatan di Malaysia menghadapi masalah kekurangan tenaga kerja yang kritikal, khususnya tenaga pakar disebabkan peletakan jawatan oleh doktor dan pakar perubatan untuk memasuki sektor swasta serta berkembangnya perkhidmatan kesihatan dan perubatan. Setelah menyedari  hakikat ini, para pelajar akan lebih berminat untuk menceburi bidang perubatan kerana pameran kerjaya yang dilaksanakan amat membantu memberikan pengetahuan am tentang kerjaya ini'
pprint(string_karangan)

('Selain itu, pameran kerjaya membantu para pelajar menentukan kerjaya yang '
 'akan diceburi oleh mereka. Seperti yang kita ketahui, pasaran kerjaya di '
 'Malaysia sangat luas dan masih banyak sektor pekerjaan di negara ini yang '
 'masih kosong kerana sukar untuk mencari tenaga kerja yang benar-benar '
 'berkelayakan. Sebagai contohnya, sektor perubatan di Malaysia menghadapi '
 'masalah kekurangan tenaga kerja yang kritikal, khususnya tenaga pakar '
 'disebabkan peletakan jawatan oleh doktor dan pakar perubatan untuk memasuki '
 'sektor swasta serta berkembangnya perkhidmatan kesihatan dan perubatan. '
 'Setelah menyedari  hakikat ini, para pelajar akan lebih berminat untuk '
 'menceburi bidang perubatan kerana pameran kerjaya yang dilaksanakan amat '
 'membantu memberikan pengetahuan am tentang kerjaya ini')


In [14]:
%%time

pprint(model.generate([string_news1, string_news2, string_news3, string_karangan], 
                      to_lang = 'en', max_length = 1000))

['TANGKAK - Tan Sri Muhyiddin Yassin said, he does not want to touch on '
 'political issues at the moment, but instead wants to focus on the welfare of '
 "the people as well as efforts to revive the country's economy affected by "
 'the Covid-19 pandemic. The Prime Minister explained the matter when speaking '
 'at the Meeting of Leaders with community leaders of the Gambir State '
 'Legislative Assembly (DUN) at the Bukit Gambir Multipurpose Hall today.',
 'ALOR SETAR - The Pakatan Harapan (PH) political turmoil is not over when it '
 'still fails to finalize the Prime Minister candidate agreed to together. Sik '
 'Member of Parliament, Ahmad Tarmizi Sulaiman said, in that regard, his party '
 'suggested that former Chairman of Parti Pribumi Bersatu Malaysia (Bersatu), '
 'Tun Dr Mahathir Mohamad and the President of Parti Keadilan Rakyat (PKR), '
 'Datuk Seri Anwar Ibrahim resign from politics as a solution.',
 'Senior Minister (Security Cluster) Datuk Seri Ismail Sabri Yaakob said

In [15]:
%%time

pprint(old_model.generate([string_news1, string_news2, string_news3, string_karangan],
                                 from_lang = 'ms', to_lang = 'en', max_length = 1000))

['TANGKAK - Tan Sri Muhyiddin Yassin said he did not want to touch on '
 'political issues for now, but instead wanted to focus on the welfare of the '
 "people as well as efforts to revive the country's economy which was affected "
 'by the Covid-19 pandemic. The Prime Minister explained the matter when '
 "speaking at the Leaders' Meeting with community leaders of the Gambir State "
 'Legislative Assembly (DUN) at the Bukit Gambir Multipurpose Hall today.',
 'ALOR SETAR - The Pakatan Harapan (PH) political crisis has not ended when it '
 'still fails to finalize the Prime Minister candidate who was mutually agreed '
 'upon. Sik Member of Parliament, Ahmad Tarmizi Sulaiman said, in this regard, '
 'he suggested that the former Chairman of Parti Pribumi Bersatu Malaysia '
 '(Bersatu), Tun Dr Mahathir Mohamad and the President of Parti Keadilan '
 'Rakyat (PKR), Datuk Seri Anwar Ibrahim resign from politics as a solution.',
 'Senior Minister (Security Cluster) Datuk Seri Ismail Sabri Ya

### Able to translate code

In [17]:
s = """
`Untuk menggunakan numpy, pertama, Anda perlu menginstalnya melalui pip. Anda dapat melakukannya dengan menjalankan perintah `pip install numpy` di terminal Anda.
Setelah numpy terinstal, Anda dapat mengimpor modul numpy dengan menambahkan baris `import numpy as np` di awal program Anda.
Berikut adalah contoh beberapa operasi dasar numpy:
``` python
import numpy as np
# membuat array numpy dari list
my_list = [1, 2, 3, 4, 5]
my_array = np.array(my_list)
# membuat array numpy dengan rentang nilai tertentu
my_range = np.arange(0, 10, 2) # nilai awal, nilai akhir, dan loncatan
# membuat array numpy dengan nilai acak
my_random_array = np.random.rand(3, 3) # 3 baris dan 3 kolom
# mengakses elemen array numpy
print(my_array[0]) # mengakses elemen pertama
# melakukan operasi matematika pada array numpy
my_array = my_array + 1 # menambah setiap elemen dengan 1
my_array = my_array * 2 # mengalikan setiap elemen dengan 2
# mengubah bentuk array numpy
my_array = np.reshape(my_array, (2, 5)) # menjadi array 2D dengan 2 baris dan 5 kolom
```
Itulah beberapa operasi dasar numpy. Anda dapat menemukan dokumentasi resmi numpy di https://numpy.org/doc/stable/.
"""

print(model.generate([s], to_lang = 'ms', max_length = 1000)[0])

`Untuk menggunakan numpy, pertama, anda perlu memasangnya melalui pip. Anda boleh melakukannya dengan menjalankan perintah `pip install numpy` di terminal anda.
Setelah numpy dipasang, anda boleh mengimport modul numpy dengan menambahkan baris `import numpy as np` pada permulaan program anda.
Berikut adalah contoh beberapa operasi asas numpy:
``` python
import numpy as np
# membuat array numpy dari senarai
my_list = [1, 2, 3, 4, 5]
my_array = np.array(my_list)
# membuat array numpy dengan julat nilai tertentu
my_range = np.arange(0, 10, 2) # nilai awal, nilai akhir, dan loncatan
# membuat array numpy dengan nilai rawak
my_random_array = np.random.rand(3, 3) # 3 baris dan 3 lajur
# mengakses elemen array numpy
print(my_array[0]) # mengakses elemen pertama
# melakukan operasi matematik pada array numpy
my_array = my_array + 1 # menambah setiap elemen dengan 1
my_array = my_array * 2 # mendarab setiap elemen dengan 2
# mengubah bentuk array numpy
my_array = np.reshape(my_array, (2, 5)) # 

### Able to maintain input structure

In [20]:
s = """
\n26 DR.27.10.2021 \n\n \nUSUL MENANGGUHKAN MESYUARAT  \n\nDI BAWAH P.M. 18(1) \n \n\nPENJUALAN ASET GAS PETRONAS DI AZERBAIJAN \n\n \n\n11.33 pg. \n\nDato’ Seri Anwar bin Ibrahim [Port Dickson]: Tuan Yang di-Pertua, saya \nmohon mengemukakan usul di bawah Peraturan Mesyuarat 18(1) dan 18(2) Peraturan-\nperaturan Majlis Mesyuarat Dewan Rakyat seperti berikut: \n\n“Bahawa Dewan yang bersidang pada hari ini merundingkan \nlaporan bahawa Petronas telah menjual aset gas di Azerbaijan dengan \nharga hampir RM10 bilion berikutan desakan kerajaan menaikkan \nbayaran dividen Petronas sebanyak RM25 bilion kepada kerajaan bagi \ntahun 2021.” \n\n \nPerkara ini adalah perkara tertentu kerana penjualan aset gas di Azerbaijan \n\nbernilai RM10 bilion dan juga pembayaran dividen kepada kerajaan sebanyak RM25 bilion \nbagi tahun 2021 adalah berbahaya dan menjejaskan masa depan Petronas. Terkait \ndengan― sedikit penjelasan.  Kerana tindakan seperti ini telah menyebabkan J.P. Morgan \nmengeluarkan Petronas dari ESG Emerging Market Bond Index dan ESG Asia Credit \nIndex.  \n\nOleh itu, perkara itu perlu disegerakan kerana Petronas mengalami defisit \nberikutan pembayaran dividen berlebihan sejak tahun 2018 sehingga kini yang memaksa \nPetronas menjual aset atau meminjam lebih banyak dana luar negara bagi menampung \naliran tunai.   \n\nIni diperkukuh dengan kenyataan Tengku Muhammad Taufik, CEO Petronas, “If \nthere is a policy shift but it comes too suddenly or too slowly, not only Petronas but the \nentire Malaysian economy could be put at risk, with regards to our energy mix.” \n\nIni perkara berkenaan dengan kepentingan orang ramai, malah seluruh ekonomi \nnegara, kerana Petronas adalah institusi penting ekonomi negara dan sewajarnya \ndiperkukuh kekuatannya dan tidak dilemahkan penarafan syarikat. Kemampuan untuk \nmelabur bagi menjana keuntungan masa depan yang mampu untuk membayar dividen \nyang munasabah kepada kerajaan demi kepentingan rakyat. Dengan tambahan RM7 \nbilion tahun ini, hasil Petronas akan berjumlah RM44.8 bilion atau 18 peratus daripada \npendapatan kerajaan bagi tahun 2021 daripada RM37.8 bilion sebelumnya. \n\nJadi, mohon persetujuan Tuan Yang di-Pertua untuk dibahaskan. [Tepuk] \n\nTuan Yang di-Pertua: Terima kasih Yang Berhormat. Sebenarnya Yang \nBerhormat perlu melengkapkan teks Yang Berhormat dan cuma dibaca teks tersebut \nmengikut peraturan. Akan tetapi, tak apa.  \n\nAhli-ahli Yang Berhormat, saya telah menerima satu pemberitahu usul di bawah \nPeraturan Mesyuarat 18(1) oleh Yang Berhormat Port Dickson, Ketua Pembangkang pada \nhari Isnin, 25 Oktober 2021. Teks usul itu adalah seperti yang dibacakan tadi dengan \nsedikit tambahan oleh Ahli Yang Berhormat Port Dickson sebentar tadi.  \n\nBagi membolehkan perkara ini ditimbangkan oleh Majlis Mesyuarat, saya \nhendaklah berpuas hati bahawa perkara yang dibangkitkan oleh Yang Berhormat itu \nmematuhi tiga syarat seperti mana biasalah iaitu: \n\n(i) perkara tertentu; \n\n(ii) bagi kepentingan orang ramai; dan \n\n(iii) berkehendak disegerakan. \n\n\n
"""
print(model.generate([s], to_lang = 'en', max_length = 1000)[0])

26 DR.27.10.2021

 
PROPOSAL ADJOURNMENT MEETING

UNDER P.M. 18(1)
 

PETRONAS GAS ASET RESULTS IN AZERBAIJAN

 

11.33 a.m.

Dato Seri Anwar bin Ibrahim [Port Dickson]: Mr. Speaker, I
beg to submit a motion under Meeting Rules 18(1) and 18(2) Rules-
rules of the Dewan Rakyat Council as follows:

"That the House sitting today negotiates
reports that Petronas has sold gas assets in Azerbaijan with
prices of almost RM10 billion following the government's insistence to raise
Petronas' dividend payment of RM25 billion to the government for
2021."

 
This matter is a certain matter because of the sale of gas assets in Azerbaijan

worth RM10 billion and also the payment of dividends to the government of RM25 billion
for 2021 it is dangerous and affects Petronas' future. Related
with a little explanation. Because actions like this have caused J.P. Morgan
to remove Petronas from the ESG Emerging Market Bond Index and ESG Asia Credit
Index. 

Therefore, the matter needs to be expedited because 

Compared to old model,

In [22]:
s = """
\n26 DR.27.10.2021 \n\n \nUSUL MENANGGUHKAN MESYUARAT  \n\nDI BAWAH P.M. 18(1) \n \n\nPENJUALAN ASET GAS PETRONAS DI AZERBAIJAN \n\n \n\n11.33 pg. \n\nDato’ Seri Anwar bin Ibrahim [Port Dickson]: Tuan Yang di-Pertua, saya \nmohon mengemukakan usul di bawah Peraturan Mesyuarat 18(1) dan 18(2) Peraturan-\nperaturan Majlis Mesyuarat Dewan Rakyat seperti berikut: \n\n“Bahawa Dewan yang bersidang pada hari ini merundingkan \nlaporan bahawa Petronas telah menjual aset gas di Azerbaijan dengan \nharga hampir RM10 bilion berikutan desakan kerajaan menaikkan \nbayaran dividen Petronas sebanyak RM25 bilion kepada kerajaan bagi \ntahun 2021.” \n\n \nPerkara ini adalah perkara tertentu kerana penjualan aset gas di Azerbaijan \n\nbernilai RM10 bilion dan juga pembayaran dividen kepada kerajaan sebanyak RM25 bilion \nbagi tahun 2021 adalah berbahaya dan menjejaskan masa depan Petronas. Terkait \ndengan― sedikit penjelasan.  Kerana tindakan seperti ini telah menyebabkan J.P. Morgan \nmengeluarkan Petronas dari ESG Emerging Market Bond Index dan ESG Asia Credit \nIndex.  \n\nOleh itu, perkara itu perlu disegerakan kerana Petronas mengalami defisit \nberikutan pembayaran dividen berlebihan sejak tahun 2018 sehingga kini yang memaksa \nPetronas menjual aset atau meminjam lebih banyak dana luar negara bagi menampung \naliran tunai.   \n\nIni diperkukuh dengan kenyataan Tengku Muhammad Taufik, CEO Petronas, “If \nthere is a policy shift but it comes too suddenly or too slowly, not only Petronas but the \nentire Malaysian economy could be put at risk, with regards to our energy mix.” \n\nIni perkara berkenaan dengan kepentingan orang ramai, malah seluruh ekonomi \nnegara, kerana Petronas adalah institusi penting ekonomi negara dan sewajarnya \ndiperkukuh kekuatannya dan tidak dilemahkan penarafan syarikat. Kemampuan untuk \nmelabur bagi menjana keuntungan masa depan yang mampu untuk membayar dividen \nyang munasabah kepada kerajaan demi kepentingan rakyat. Dengan tambahan RM7 \nbilion tahun ini, hasil Petronas akan berjumlah RM44.8 bilion atau 18 peratus daripada \npendapatan kerajaan bagi tahun 2021 daripada RM37.8 bilion sebelumnya. \n\nJadi, mohon persetujuan Tuan Yang di-Pertua untuk dibahaskan. [Tepuk] \n\nTuan Yang di-Pertua: Terima kasih Yang Berhormat. Sebenarnya Yang \nBerhormat perlu melengkapkan teks Yang Berhormat dan cuma dibaca teks tersebut \nmengikut peraturan. Akan tetapi, tak apa.  \n\nAhli-ahli Yang Berhormat, saya telah menerima satu pemberitahu usul di bawah \nPeraturan Mesyuarat 18(1) oleh Yang Berhormat Port Dickson, Ketua Pembangkang pada \nhari Isnin, 25 Oktober 2021. Teks usul itu adalah seperti yang dibacakan tadi dengan \nsedikit tambahan oleh Ahli Yang Berhormat Port Dickson sebentar tadi.  \n\nBagi membolehkan perkara ini ditimbangkan oleh Majlis Mesyuarat, saya \nhendaklah berpuas hati bahawa perkara yang dibangkitkan oleh Yang Berhormat itu \nmematuhi tiga syarat seperti mana biasalah iaitu: \n\n(i) perkara tertentu; \n\n(ii) bagi kepentingan orang ramai; dan \n\n(iii) berkehendak disegerakan. \n\n\n
"""
print(old_model.generate([s], from_lang = 'ms', to_lang = 'en', max_length = 1000)[0])

26 DR.27.10.2021 MOTION TO ADJOURN THE MEETING UNDER P.M. 18(1) JUALAN OF PETRONAS GAS ASET IN AZERBAIJAN 11.33 a.m. Dato Seri Anwar bin Ibrahim [Port Dickson]: Mr. Speaker, I beg to submit a motion under Meeting Rule 18(1) and 18(2) of the Rules of the Dewan Rakyat Meeting Council as follows: "That the House sitting today negotiates a report that Petronas has sold gas assets in Azerbaijan at a price of almost RM10 billion following the government's insistence on raising Petronas' dividend payment of RM25 billion to the government for 2021." This is a certain matter because the sale of gas assets in Azerbaijan worth RM10 billion and also the payment of dividends to the government amounting to RM25 billion for 2021 is dangerous and affects the future of Petronas. Related to a little explanation. Because actions like this have caused J.P. Morgan to remove Petronas from the ESG Emerging Market Bond Index and ESG Asia Credit Index. Therefore, the matter needs to be expedited because Petron

### Able to translate indonesian, banjarese and javanese

In [23]:
s1 = 'Kita ora duwe wektu kanggo gelut utawa sengit.'
s2 = 'Tiga dekade kemudian area sekolah diperluas.'
s3 = 'Maklum haja lah muntung guri wadi kawa haja dipisiti, amun muntung manusia siapang nang hingkat manduga.'

In [25]:
pprint(model.generate([s1, s2, s3], to_lang = 'en', max_length = 1000))

["We don't have time to fight or get intense.",
 'Three decades later the school area was expanded.',
 'It is understood that the smug is a good person to be shunned, but the smug '
 'of humans is ready to be shunned.']


In [26]:
pprint(model.generate([s1, s2, s3], to_lang = 'ms', max_length = 1000))

['Kita tidak ada masa untuk bergaduh atau garang.',
 'Tiga dekad kemudian kawasan sekolah diperluaskan.',
 '"Maklumlah hanya menyapa orang yang tidak tahu apa yang mereka lakukan, '
 'tetapi manusia yang tidak tahu apa yang mereka lakukan."']
